In [ ]:
rasa==3.6.15
streamlit==1.29.0
python-telegram-bot==20.3
pandas==2.0.3
numpy==1.24.3
python-dotenv==1.0.0

In [ ]:
version: "3.1"

nlu:
- intent: greet
  examples: |
    - hi
    - hello
    - hey there
    - good morning
    - greetings

- intent: goodbye
  examples: |
    - bye
    - goodbye
    - see you later
    - have a nice day
    - talk to you soon

- intent: order_status
  examples: |
    - where is my order?
    - status of my package
    - when will my order arrive?
    - tracking number [AB12345](order_number)
    - has my order shipped?

- intent: refund_request
  examples: |
    - I want a refund
    - how do I get my money back?
    - return my product
    - refund status
    - cancel my order

- intent: product_issue
  examples: |
    - the product is damaged
    - item arrived broken
    - wrong item received
    - missing parts
    - defective product

- intent: account_help
  examples: |
    - I can't log in
    - password reset
    - account locked
    - forgot my username
    - change email address

In [ ]:
version: "3.1"

stories:
- story: greet and goodbye
  steps:
  - intent: greet
  - action: utter_greet
  - intent: goodbye
  - action: utter_goodbye

- story: order status inquiry
  steps:
  - intent: order_status
  - action: utter_ask_order_number
  - intent: inform
    entities:
    - order_number
  - action: action_check_order_status

- story: refund request
  steps:
  - intent: refund_request
  - action: utter_ask_refund_details
  - intent: inform
  - action: action_process_refund

- story: product issue
  steps:
  - intent: product_issue
  - action: utter_ask_product_details
  - intent: inform
  - action: action_process_product_issue

- story: account help
  steps:
  - intent: account_help
  - action: utter_account_help

In [ ]:
version: "3.1"

rules:
- rule: greet
  steps:
  - intent: greet
  - action: utter_greet

- rule: goodbye
  steps:
  - intent: goodbye
  - action: utter_goodbye

In [ ]:
version: "3.1"

intents:
  - greet
  - goodbye
  - order_status
  - refund_request
  - product_issue
  - account_help
  - inform

entities:
  - order_number

responses:
  utter_greet:
    - text: "Hello! Welcome to Future Interns customer support. How can I help you today?"

  utter_goodbye:
    - text: "Goodbye! Have a great day!"
    - text: "Thanks for contacting us. See you soon!"

  utter_ask_order_number:
    - text: "I can check your order status. Could you please provide your order number?"

  utter_ask_refund_details:
    - text: "I can help with your refund request. Please share your order number and reason for refund."

  utter_ask_product_details:
    - text: "I'm sorry to hear about the issue with your product. Could you share your order number and describe the problem?"

  utter_account_help:
    - text: "For account-related issues, please visit our support page at support.futureinterns.com or provide your username so I can assist further."

actions:
  - action_check_order_status
  - action_process_refund
  - action_process_product_issue

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true

In [ ]:
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
import random

class ActionCheckOrderStatus(Action):
    def name(self) -> Text:
        return "action_check_order_status"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        order_number = next(tracker.get_latest_entity_values("order_number"), None)

        if order_number:
            status = random.choice(["shipped", "processing", "out for delivery", "delivered"])
            dispatcher.utter_message(text=f"Your order {order_number} is currently {status}.")
        else:
            dispatcher.utter_message(text="I couldn't find your order number. Please check and provide it again.")

        return []

class ActionProcessRefund(Action):
    def name(self) -> Text:
        return "action_process_refund"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        dispatcher.utter_message(text="I've submitted your refund request. Our team will process it within 3-5 business days.")
        return []

class ActionProcessProductIssue(Action):
    def name(self) -> Text:
        return "action_process_product_issue"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        dispatcher.utter_message(text="I've logged your product issue. Our support team will contact you within 24 hours to resolve this.")
        return []

In [ ]:
import streamlit as st
from rasa.core.agent import Agent
from rasa.shared.constants import DEFAULT_MODELS_PATH
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Load trained Rasa model
model_path = DEFAULT_MODELS_PATH
agent = Agent.load(model_path)

st.title("Future Interns Customer Support Chatbot")
st.caption("Built for Machine Learning Internship Task 3")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("How can I help you today?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Get response from Rasa
    async def get_rasa_response():
        responses = await agent.handle_text(prompt)
        return responses[0]['text'] if responses else "I didn't understand that. Can you rephrase?"

    response = asyncio.run(get_rasa_response())

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

# Sidebar with information
with st.sidebar:
    st.header("About This Chatbot")
    st.write("""
    This chatbot was developed as part of the Future Interns Machine Learning Internship (Task 3).

    It demonstrates:
    - Natural Language Understanding (NLU)
    - Dialogue Management
    - Integration with Streamlit and Telegram

    The chatbot can handle:
    - Order status inquiries
    - Refund requests
    - Product issues
    - Account assistance
    """)

    st.markdown("[GitHub Repository](https://github.com/yourusername/FUTURE_ML_03)")

In [ ]:
import os
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from rasa.core.agent import Agent
from dotenv import load_dotenv
import asyncio

# Load environment variables
load_dotenv()
TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")

# Load Rasa agent
agent = Agent.load("models/")

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("""
    Hello! I'm the Future Interns Customer Support Bot.
    I can help you with:
    - Order status inquiries
    - Refund requests
    - Product issues
    - Account assistance

    How can I help you today?
    """)

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("""
    Here's what I can do:

    • Ask about your order status
    • Process refund requests
    • Report product issues
    • Help with account problems

    Just type your question and I'll do my best to help!
    """)

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text

    # Get response from Rasa
    responses = await agent.handle_text(text)
    response = responses[0]['text'] if responses else "I didn't understand that. Can you rephrase?"

    await update.message.reply_text(response)

def main():
    print("Starting bot...")
    app = Application.builder().token(TELEGRAM_TOKEN).build()

    # Commands
    app.add_handler(CommandHandler("start", start_command))
    app.add_handler(CommandHandler("help", help_command))

    # Messages
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Run bot
    print("Polling...")
    app.run_polling(poll_interval=3)

if __name__ == "__main__":
    main()